In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [5]:
import re

In [15]:
input_file = "sheep-data/input2.txt"
file = open(input_file, "r") 
text = file.read() 
print('corpus length:', len(text))

corpus length: 464826


In [17]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 84


In [18]:
## Chopping the text up into strings of 40 characters to train on
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 154929


In [19]:
## Vectorizing our sequences
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [20]:
## We're designing our model here- choosing the number of layers and such
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [21]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Showing the poetry quality at the end of each epoch
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        ## What we're doing is giving the bot 40 characters and asking
        ## it to predict the 41st character, over and over
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [22]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Here's where we train the model on our data
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
154880/154929 [============================>.] - ETA: 0s - loss: 2.1938
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "eenhouse where we'll see a green heron
i"
eenhouse where we'll see a green heron
in the with the sen the word and the sear the sheep
The was the word the world the see the stree the word the mong the the shore the word the word the word the the the wark the wark the word the way the the the sen the sen the strow the word the his down the rese the sen the sang the sen the the stree the word the count the word the with the world the spree the sen the the the with the were the wor
----- diversity: 0.5
----- Generating with seed: "eenhouse where we'll see a green heron
i"
eenhouse where we'll see a green heron
is the row the were, 
The would strees the moon be the light the mine in the strow the wath shee 
Some the world where the tore the word light and the see the shalf not for the wind of the wear the more
whose the for

In single file they mogenn -snow,
And I had frinn their shord! Rusted and prinn.
The swoed him gigness, rudants down snow crifts
Above itselp nivel in palt joy when the went is
and I the folling, to knswapling for sord,
pard? Add the faces mole when three fall thriftly of welly
hull, grand the was thon in hur
girly joy, I came his sanges
along that hors, with work a piet to cometel,
Tholl stalle and bream insames st
----- diversity: 1.2
----- Generating with seed: "n they themselves
In single file they m"
n they themselves
In single file they maybing's eal
To thourol.
Godiant; to fexping that glistenpow to beet hearty,
his roonee, bund wallh oct,
Rarn Is down orropers
Stampting bundure of siketeasion fewrater for mist 'sing Chon thing:
Terh dalable dow his hine, spendshool.
Ruste winh in vife wore were
exlangward-nablues by walk Alres.urur
aak'd what my ymed from God'd 
While day, :'d lapiest. Dawsquir, so do sluli's by 
Oh bown tryfe
154929/154929 [==============================] - 23

recollecting the story her see and the strange the parts the strain.
The stars of the train the stars the the the from the dead the day 
The wind the wind the strain; 
And the small see the work in the street the the sea 
And the wind start the stars in the strain.
The wind the strains to the stars beare and the strain.
The world and the spart of the strain:
And the stars the seemer the strain.
And the start t
----- diversity: 0.5
----- Generating with seed: "raining their own shadows;
recollecting "
raining their own shadows;
recollecting out the strets of here in the grave on the trast like land and the dream,
The fither his horse to mears my penplenent have the tablement in the see.
In the pro-genty verse mountains not read.
And his hones with the beaten the ride and the brought of the street.
And the mountains the start, or the came for a farmor.
Some sead the strengted on the lay the sea 
And was of the down on the channer, 
Th
----- diversity: 1.0
----- Generating with seed: "rai

The stame he didge: : his stands or to quipedanks munchering.
When she own gisians for the skeak of when I keptrosion whichnars manys Wast 
Distante of white Table of silverly
's lace where me their thy porter and head, 
So apquelefly none coon, moan train would to all thy the other.
peach you fursters and kick a wing'd the things noon vigoing,
Ih lescelt
----- diversity: 1.2
----- Generating with seed: "
Delight our souls with talk of knightly"

Delight our souls with talk of knightly, of weagles preatate fad
in year, like wonwe stood. was pesperation
colvan-sam bats for the days
Strommer trempen gacded dacky came,
Ald flower, indetion. The Aurcitaus, fall,
And bear Fourdeding a born,
a yiuts,
Hadked rifleas, julg, whaling lip to never live the wart, 
Way 'ariences up.
Somefrotherfoldad range of boke waving heroer repored; 
That Goodal, he you sunca Notioos buch'm, and bea
154929/154929 [==============================] - 258s 2ms/step - loss: 1.6418
Epoch 12/60
154880/154929 [========

Stuck in the gray ocean, surrounded by the stars.
The see the stars of the stars of the sea
to be the stars the morning and the sea
to built the stars and the stars of the sea
and the wind and the story and the stars.
The see the stars the sheep the sheep, the stream. 
The book of the perple of the book of the seemed the stars.
The stars the wind the stars the stars to the sea
to bleec the story of the stars of the blood,
           
----- diversity: 0.5
----- Generating with seed: "n
Stuck in the gray ocean, surrounded by"
n
Stuck in the gray ocean, surrounded by see 
A start he say the park of the sangted to the cather;
                                                                                                                                                                                                                                                                                                                                                   
----- diversity: 1.0
----- Gen

...

I love the sheep and the stores and the stores and the stores and the stores 
The way to see the stores and the sun, 
The stars of the store the sheep, the shields and the stories,
the ships of the stores and the stories.
When t
----- diversity: 0.5
----- Generating with seed: "e roads,
the swallow snapping back flies"
e roads,
the swallow snapping back flies sheep,
far a stars of white broke gorse of the white bark and ground 
The lead of stories of the stories, the dear 
The god of the roofs, and do know in the stone
moon of father, the place and stone, the bore 
We can to the she was out the work, the discourse of the first come, 
So he could midds with the west the sheep,
Some round the ocean stores in the grey from the flost,
the king the mountai
----- diversity: 1.0
----- Generating with seed: "e roads,
the swallow snapping back flies"
e roads,
the swallow snapping back fliest.'
    Thy hunger for something with smile to flower their gushborniad-sloping boacel dy,
Through checked

were? That stub am a louded in not inever out, Sep, no and then datary-cetral sladds. 
Yes, and kid lived theching stiffer inside.
----- diversity: 1.2
----- Generating with seed: "re was something to find there,
To turn "
re was something to find there,
To turn a male! Frzging and uch. Iter your morning and. 

bly's regming there 
From the other foffbrancips
His locdoua tancing your neare, 
And its gloss lores into the stateriol eath;
   Fcritter thrnewing with while , but seeth 
Bulgorm, they comn tomper
One doams rark Christly their 
Watters of bright less or sectale all they; great and is intflif,
and face in else spanged, and the nerymants swimmly 
154929/154929 [==============================] - 252s 2ms/step - loss: 1.5335
Epoch 50/60
154880/154929 [============================>.] - ETA: 0s - loss: 1.5306
----- Generating text after Epoch: 49
----- diversity: 0.2
----- Generating with seed: "hattering and cussing trying to figure h"
hattering and cussing trying to figure him.
An

                         And the morning a trees of the same seating the heart and the street of the street of the street of the streams and the white 
The waveled the street of the rest the stand 
The hoos'd the didged her banks of the street and the sea
and stream and late streams of 
----- diversity: 0.5
----- Generating with seed: "some treasure-house of mighty kings, 
So"
some treasure-house of mighty kings, 
Some orm be, the can are sate with make a marm, 
And expent of the earth.
And prozzment, the stand of the shore can brief.
The sun from the storms at the sun was bucked and he dresse 
The warm of breath of screep, the with a man now seem.
The sun and watch of one strain
when the pain of the hands of the strong.
                                       black,
but the his father and the unstrocking of 
----- diversity: 1.0
----- Generating with seed: "some treasure-house of mighty kings, 
So"
some treasure-house of mighty kings, 
So hayspraye shought. Onces, on youl atone
would c

Our heaveness and the eyes:
They and as yellow heart-like too sunlight that know 
And apteneding eIgling is once 
Their gottry flash start
to the city tred by inse, 
the trought here sprung in 
----- diversity: 1.2
----- Generating with seed: "y the grief.
Clark Harrison, at nineteen"
y the grief.
Clark Harrison, at nineteen'd of quietless the renied wattles anothonger,
arting about and kills Lowicicle, as a god a dustened.
You was at they finger betweend
their art and I
tent wide a blawinges?
By lambless lined cath, yeby.
Which 
That you never been still tired on around the aupses. Then her vince dork 
Nor bliss heared erse back wide on'd . . ...ypoading flance poistes.
I red into acoze yent, the most made slake..ew
154929/154929 [==============================] - 249s 2ms/step - loss: 1.5256
Epoch 57/60
154880/154929 [============================>.] - ETA: 0s - loss: 1.5277
----- Generating text after Epoch: 56
----- diversity: 0.2
----- Generating with seed: "  "Ferood, shame bids u

And was a shart the sun is not the way of the sea;
the the storms in the barkened and the sun is not an intrees
and the sun was the sun rest, 
And the morning the street of the smoothile and sheep
The fair for the world and the sun was w
----- diversity: 0.5
----- Generating with seed: "o be the icon.
In college, he remembers
"
o be the icon.
In college, he remembers
and then works and there way and sing the share.
The four was our folded the went seed in a station 
For we have been purse with the sun is a more comes and they walks,
the distances and her the company warm,
who pillet self'd and the power repore: 
In the world the stone hor wings to the sen and light
with so a sun go were through summer-work of the bord.
In the thoughts and one birthing and the 
----- diversity: 1.0
----- Generating with seed: "o be the icon.
In college, he remembers
"
o be the icon.
In college, he remembers
 a enI feather marder, and heard,
 he milder of stilled actors. I, the gold, 
And from dark from 

In [26]:
model.save('sheepbot3.h5')

In [25]:
## Here is where you can actually use the model on a given piece of text!

for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = 'There once was a cool guy named Cam.....'
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(800):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

----- diversity: 0.2
----- Generating with seed: "There once was a cool guy named Cam....."
There once was a cool guy named Cam......
 the bestorment the stream and the street 
The warm her back and the common of the distant 
The we've we have seen the sun was a strong, 
And when the country of the shadows and the common of the wood.
The warm the shakened of the seasong and the sea
and brother was the same who come, 
And the said west the sun was the stream and the finged the sea, 
And the sun was a guess and the cornial streams and the sun for the stare 
The bed the sun was the morning and the sun is not my spitted and the common of the distant 
The field the mountains of the colored by the work,
and the sun will to the sun was a stand 
The said, and the water of the sea, 
And born the sun was the sun is not flowers of the morning of the work,
the seating the room of the stone of the works
and the warmied and the way 
----- diversity: 0.5
----- Generating with seed: "There once was a 

/home/Randus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


Beauth like the small greet, 
The resount of the shadows the conkering to my many old trees,
the bold had the mountains of every pain 
Who was the heart he says, and the rain and brown 
The pease and the sun the way the coon light of the boat.
And for the bones of bedian from a shakes the sea, 
See a callar lived it see the wain down to the wells of it,
angual of the columf in the work,
an of the flack on the flowers of the woods
The sun they said we should be and is well worsed or the pain and his rise and soul. 
And flass, or by the turfees of day of the sea
----- diversity: 1.0
----- Generating with seed: "There once was a cool guy named Cam....."
There once was a cool guy named Cam.....my pourds though to charded,
manianing we, greit is theory.
Soldy a towning, whan immortain neat-angine flower
wabun young mortale at onatio tongue use again mudes years, hie drawnce
There, in this lights that since I waking friend . .
For let many lockt trees they closer mort greetic blue, 
With str